In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# List the top level of MyDrive (quick visual check)
!ls -lh "/content/drive/MyDrive"

total 11M
-rw------- 1 root root  73K Aug 19 09:11  basic-text.pdf
drwx------ 2 root root 4.0K Mar 27  2024 'Colab Notebooks'
-rw------- 1 root root 348K Aug  3  2023  _DSC2237---01.jpg
drwx------ 4 root root 4.0K Jul 12 05:52  ROS_LAB
-rw------- 1 root root 3.2M Mar 16  2024 'Translation 13-Mar-2024 11-42-16.pdf'
-rw------- 1 root root 6.9M Aug 12 10:46  Unit-1-ccnotes.pdf


In [ ]:
!pip install PyPDF2

In [ ]:
!pip install langchain langchain-community sentence-transformers faiss-cpu pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 24.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_community.llms import HuggingFaceHub
from transformers import pipeline

In [ ]:
loader = PyPDFLoader("/content/drive/MyDrive/basic-text.pdf")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(documents)

print(f"Total Chunks Created: {len(chunks)}")
print("\n--- Sample Chunk ---\n")
print(chunks[0].page_content[:300])

Total Chunks Created: 2

--- Sample Chunk ---

Sample Document for PDF Testing
Introduction
This is a simple document created to test basic PDF functionality. It includes various text formatting
options to ensure proper rendering in PDF readers.
Text Formatting Examples
1. Bold text is used for emphasis.
2. Italic text can be used for titles or 


In [ ]:
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
db = FAISS.from_documents(chunks, embedding_model)

In [ ]:
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [ ]:
local_llm = pipeline("text2text-generation", model="google/flan-t5-small", max_new_tokens=200)

from langchain.llms import HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=local_llm)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

In [ ]:
query = "What is the main topic discussed in this PDF?"
result = qa_chain({"query": query})

print("\n🔹 Answer:")
print(result["result"])

print("\nSources:")
for doc in result["source_documents"]:
    print(f"Page {doc.metadata['page']}:\n{doc.page_content[:200]}\n")


🔹 Answer:
Sample Document for PDF Testing Introduction This is a simple document created to test basic PDF functionality. It includes various text formatting options to ensure proper rendering in PDF readers. Text Formatting Examples 1. Bold text is used for emphasis. 2. Italic text is used for titles or subtle emphasis. 3. Strikethrough is used to show deleted text. Lists Here's an example of an ordered list: 1. First item 2. Second item 3. Third item Quote This is an example of a block quote. It can be used to highlight important information or citations. Table Header 1 Header 2 Header 3 Row 1, Col 1 Row 1, Col 2 Row 1, Col 3 Row 2, Col 1 Row 2, Col 2 Row 2, Col 3 This document demonstrates various formatting options that should translate well to PDF format. This sample PDF file is provided by Sample-Files.com. Visit us for more sample files and resources.

📄 Sources:
Page 0:
Sample Document for PDF Testing
Introduction
This is a simple document created to test basic PDF functionali